In [1]:
import pandas as pd
from Bio import SeqIO, AlignIO, Seq
import numpy as np
from collections import Counter
from helper import *

### Preprocessing

#### Creating csv file from the data in Jaffe et al. and cleaning sequence ID from ":"

The first thing we need to do is to remove places with more than one `_` from the ids because after moving to iTOL this will be a problem. We do this for the files `jaffe_et_al_2018_rubisco_superfamilies.faa`, `RuBisCO.300-700.faa`, and `milo_synthetized_rubisco.faa`

In [4]:
sequences = []
for record in SeqIO.parse('../data/jaffe_et_al_2018_rubisco_superfamilies.faa', "fasta"):
    record.description = record.description.split(':')[0]
    record.id = record.id.split(':')[0]
    record.name = record.name.split(':')[0]
    sequences.append(record)
!mkdir -p ../output/01_70p_tree
with open(r"../output/01_70p_tree/jaffe_et_al_2018_rubisco_superfamilies_clean.faa", "w") as output_handle:
    SeqIO.write(sequences, output_handle, "fasta")

In [5]:
sequences = []
for record in SeqIO.parse('../data/RuBisCO.300-700.faa', "fasta"):
    record.seq = record.seq.rstrip(chars='*')
    sequences.append(record)
        
with open(r"../output/01_70p_tree/RuBisCO.300-700_clean.faa", "w") as output_handle:
    SeqIO.write(sequences, output_handle, "fasta")

### Merge fasta files and cluster sequences

In [6]:
#!cat ../data/milo_synthetized_rubisco.faa ../data/RuBisCO.300-700.faa ../data/flamholz_et_al_2019_kinetically_characterized.faa ../data/erb_et_al_2012_rubisco_superfamilies.faa > ../data/merged_data.faa
!cat ../data/milo_synthetized_rubisco.faa ../output/01_70p_tree/RuBisCO.300-700_clean.faa ../data/flamholz_et_al_2019_kinetically_characterized.faa ../output/01_70p_tree/jaffe_et_al_2018_rubisco_superfamilies_clean.faa > ../output/01_70p_tree/merged_data.faa

### remove sequences with ambiguous calls 

In [7]:
aa = ['A','R','N','D','C','G','Q','E','H','I','L','K','M','F','P','S','T','W','Y','V']
ambiguous_aa = ['B','Z','X','J']
canon = []
for record in SeqIO.parse('../output/01_70p_tree/merged_data.faa', "fasta"):
    no_of_aa = np.array([x in ambiguous_aa for x in record.seq]).sum()
    if no_of_aa == 0:
        canon.append(record)

with open(r"../output/01_70p_tree/merged_data_clean.faa", "w") as output_handle:
    SeqIO.write(canon, output_handle, "fasta")

### Cluster sequences using uclust

In [8]:
!../bin/usearch11.0.667_i86linux32 -cluster_fast ../output/01_70p_tree/merged_data_clean.faa -id 0.7 -uc ../output/01_70p_tree/uclust_all_0.7.uc

usearch v11.0.667_i86linux32, 4.0Gb RAM (16.3Gb total), 8 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: yinonmoise.baron@weizmann.ac.il

00:01 73Mb    100.0% Reading ../output/01_70p_tree/merged_data_clean.faa
00:01 112Mb   100.0% DF
00:01 113Mb  49110 seqs, 48052 uniques, 47400 singletons (98.6%)
00:01 113Mb  Min size 1, median 1, max 48, avg 1.02
00:01 120Mb   100.0% DB
00:04 181Mb   100.0% 591 clusters, max size 14732, avg 83.1
                                                           
      Seqs  48052 (48.1k)
  Clusters  591
  Max size  14732 (14.7k)
  Avg size  83.1
  Min size  1
Singletons  241, 0.5% of seqs, 40.8% of clusters
   Max mem  181Mb
      Time  3.00s
Throughput  16.0k seqs/sec.



### Take only cluster fasta files and create csv file

In [9]:
parse_uclust(infile='../output/01_70p_tree/uclust_all_0.7.uc',
             fasta='../output/01_70p_tree/merged_data_clean.faa',
             outfasta='../output/01_70p_tree/uclust_all_0.7.faa',
             outfile='../output/01_70p_tree/uclust_all_0.7.csv'
            )

## Create multiple sequence alignment

In [10]:
!../bin/mafft-linux64/mafft.bat ../output/01_70p_tree/uclust_all_0.7.faa > ../output/01_70p_tree/uclust_all_0.7.aln

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  501 / 591
done.

Constructing a UPGMA tree (efffree=0) ... 
  580 / 591
done.

Progressive alignment 1/2... 
STEP   398 / 590  h
Reallocating..done. *alloclen = 2429
STEP   501 / 590 
Reallocating..done. *alloclen = 3575

done.

Making a distance matrix from msa.. 
  500 / 591
done.

Constructing a UPGMA tree (efffree=1) ... 
  580 / 591
done.

Progressive alignment 2/2... 
STEP   388 / 590  h
Reallocating..done. *alloclen = 2432
STEP   501 / 590 
Reallocating..done. *alloclen = 3436

done.

disttbfast (aa) Version 7.427
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The def

## Clean MSA to contain only positions with more than 5% coverage (based on Jaffe et al. 2018)

In [11]:
clean_aln(infile='../output/01_70p_tree/uclust_all_0.7.aln',
          outfile='../output/01_70p_tree/uclust_all_0.7_trimmed.aln')

### Run RaxML on CIPRES

In [ ]:
cip = CipresClient.Client(appname='RO',
                    appID='rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA',
                    baseUrl='https://cipresrest.sdsc.edu/cipresrest/v1',
                    username='yinonbaron',
                    password='Mchcav11~')


In [ ]:

job = cip.submitJob(vParams={'toolId': 'RAXMLHPC8_REST_XSEDE',
                       'datatype_': 'protein',
                       'runtime_': '160',
                       'select_analysis_': 'fa',
                       'choose_bootstrap_': 'x',
                       'printbrlength_': '1'},
              inputParams={'infile_': '../output/01_70p_tree/uclust_all_0.7_trimmed.aln'},
              metadata={'statusEmail':'true'})

In [ ]:
if cip.getJobStatus(jobHandle=job.jobHandle).isDone():
    !mkdir -p ../output/02_90p_autotrophic_rubisco_tree/RaxML/
    job.downloadResults('../output/01_70p_tree//RaxML/')
else:
    print('Job ' + job.jobHandle + ' not finished')

In [3]:
#!curl -u yinonbaron:Mchcav11~ -H cipres-appkey:rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA https://cipresrest.sdsc.edu/cipresrest/v1/job/yinonbaron -F tool=RAXMLHPC8_REST_XSEDE  -F input.infile_=@../output/01_70p_tree/uclust_all_0.7_trimmed.aln -F metadata.statusEmail=true -F vparam.datatype_=protein -F vparam.runtime_=160 -F  vparam.select_analysis_=fa -F vparam.choose_bootstrap_=x -F vparam.printbrlength_=1

In [3]:
# Optional - create tree using fasttree
#!../bin/FastTree -spr 4 -mlacc 2 -slownni ../output/03_uclust_all_0.7_trimmed.aln > ../output/04_uclust_all_0.7.nwk